In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from typing import Any, Dict, List, Optional, Tuple

import hydra
import lightning as L
import rootutils
import torch
import time
from lightning import Callback, LightningDataModule, LightningModule, Trainer
from lightning.pytorch.loggers import Logger
from omegaconf import DictConfig
import pickle
# from src.data.scared_datamodule
root = rootutils.setup_root('./', indicator=".project-root", pythonpath=True)
print(root)

/bd_byta6000i0/users/surgical_depth/surgical_depth


In [21]:
from src.data import SCAREDDataModule
from src.models.components import resnet18, DinoEncoder, R2Plus1D, DPTHead
# from src.models.components.depth_decoder import Bottleneck

In [15]:
dm = SCAREDDataModule('/bd_byta6000i0/users/surgical_depth/SCARED_fixed/scared.hdf5','../data', num_workers=8)

In [16]:
dm.prepare_data()
dm.setup()

In [17]:
start_time = time.time()
for batch_idx, batch in enumerate(dm.val_dataloader()):
    print(time.time() - start_time)
    start_time = time.time()
    my_batch = batch
    # break

2.9376704692840576
3.753941774368286
0.0008475780487060547
0.0008325576782226562
0.0007703304290771484
0.0001723766326904297
0.00011348724365234375
0.0001418590545654297
0.26516222953796387
0.5473334789276123
0.2808966636657715
1.2433109283447266
0.0006034374237060547
0.0006031990051269531
0.0005290508270263672
0.00014662742614746094
0.22575831413269043
1.0324547290802002
0.0005602836608886719
0.18122577667236328
0.2020559310913086
8.916854858398438e-05
0.00015234947204589844
0.00034308433532714844
0.18154168128967285
0.1582012176513672
0.19475007057189941
0.4562406539916992
0.21610355377197266
0.22164082527160645
0.20576977729797363
0.00031185150146484375
0.21717429161071777


KeyboardInterrupt: 

In [8]:
list(my_batch)

['color_clip', 'depth_gt_clip']

In [14]:
my_batch['color_clip'].max()

tensor(2.6400)

In [4]:
batch = torch.randn((12,3,5,224,280))

In [6]:
split_batch = torch.chunk(batch, batch.shape[-3], 2)

In [8]:
for i in split_batch:

    print(i.squeeze(2).shape)

torch.Size([12, 3, 224, 280])
torch.Size([12, 3, 224, 280])
torch.Size([12, 3, 224, 280])
torch.Size([12, 3, 224, 280])
torch.Size([12, 3, 224, 280])


In [3]:
my_batch['depth_gt_clip'].shape

NameError: name 'my_batch' is not defined

In [1]:
split_tensors = torch.chunk(my_batch['depth_gt_clip'], 5, dim=2)

split_tensors

NameError: name 'torch' is not defined

In [42]:
with open('batch.pkl', 'rb') as f:
    batch = pickle.load(f)

In [43]:
batch[('color_clip',0)]

tensor([[[[[0.7412, 0.7216, 0.7176,  ..., 0.0706, 0.0667, 0.0667],
           [0.7373, 0.7294, 0.7255,  ..., 0.0667, 0.0667, 0.0667],
           [0.7333, 0.7373, 0.7333,  ..., 0.0667, 0.0706, 0.0745],
           ...,
           [0.5059, 0.5137, 0.5216,  ..., 0.6902, 0.6980, 0.6980],
           [0.5098, 0.5059, 0.5137,  ..., 0.6784, 0.6824, 0.6784],
           [0.5020, 0.5059, 0.5059,  ..., 0.6941, 0.6863, 0.6706]],

          [[0.7059, 0.7059, 0.6980,  ..., 0.0706, 0.0667, 0.0667],
           [0.7098, 0.7020, 0.6902,  ..., 0.0667, 0.0667, 0.0667],
           [0.7098, 0.7059, 0.6902,  ..., 0.0706, 0.0706, 0.0745],
           ...,
           [0.5020, 0.5059, 0.5098,  ..., 0.6941, 0.6588, 0.6157],
           [0.5020, 0.5020, 0.5020,  ..., 0.6863, 0.6667, 0.6549],
           [0.4980, 0.4941, 0.4902,  ..., 0.6588, 0.6392, 0.6549]],

          [[0.6627, 0.6588, 0.6588,  ..., 0.0706, 0.0667, 0.0667],
           [0.6627, 0.6471, 0.6471,  ..., 0.0706, 0.0667, 0.0667],
           [0.6824, 0.6706

In [7]:
RD = R2Plus1D([2,2,2,2])

In [80]:
mm = resnet18(sample_size = 1, sample_duration = 1)

/bd_byta6000i0/users/surgical_depth/surgical_depth/src/models/components/temporal_encoder.py:145: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [17]:
with   torch.no_grad():
    result = RD(batch[('color_clip', 0)])

In [16]:
batch[('color_clip', 0)].shape[-2:]

torch.Size([224, 280])

In [18]:
for i in result:
    print(i.shape)

torch.Size([12, 96, 5, 112, 140])
torch.Size([12, 192, 5, 56, 70])
torch.Size([12, 384, 5, 28, 35])
torch.Size([12, 768, 5, 14, 18])


In [22]:
dino = DinoEncoder()

In [25]:
with torch.no_grad():
    result = dino.temporal_forward(my_batch['color_clip'])

In [26]:
for i in result:
    print(i.shape)

torch.Size([12, 96, 5, 112, 140])
torch.Size([12, 192, 5, 56, 70])
torch.Size([12, 384, 5, 28, 35])
torch.Size([12, 768, 5, 14, 18])


In [130]:
for i in result:
    print(i.shape)

torch.Size([12, 96, 112, 140])
torch.Size([12, 192, 56, 70])
torch.Size([12, 384, 28, 35])
torch.Size([12, 768, 14, 18])


In [20]:
features = dino.temporal_foward(batch)

In [21]:
for feature in features:
    print(feature.shape)

torch.Size([12, 96, 5, 112, 140])
torch.Size([12, 192, 5, 56, 70])
torch.Size([12, 384, 5, 28, 35])
torch.Size([12, 768, 5, 14, 18])


In [177]:
bo = Bottleneck()

In [179]:
bo(features)

[tensor([[[[[ 9.7808e-02,  9.7808e-02,  9.7808e-02,  ..., -2.2591e-03,
             -2.2591e-03, -2.2591e-03],
            [ 9.7808e-02,  9.7808e-02,  9.7808e-02,  ..., -2.2591e-03,
             -2.2591e-03, -2.2591e-03],
            [ 9.7808e-02,  9.7808e-02,  9.7808e-02,  ..., -2.2591e-03,
             -2.2591e-03, -2.2591e-03],
            ...,
            [ 1.1583e-01,  1.1583e-01,  1.1583e-01,  ...,  5.7965e-02,
              5.7965e-02,  5.7965e-02],
            [ 1.1583e-01,  1.1583e-01,  1.1583e-01,  ...,  5.7965e-02,
              5.7965e-02,  5.7965e-02],
            [ 1.1583e-01,  1.1583e-01,  1.1583e-01,  ...,  5.7965e-02,
              5.7965e-02,  5.7965e-02]]],
 
 
          [[[ 1.5188e-01,  1.5188e-01,  1.5188e-01,  ..., -2.0085e-01,
             -2.0085e-01, -2.0085e-01],
            [ 1.5188e-01,  1.5188e-01,  1.5188e-01,  ..., -2.0085e-01,
             -2.0085e-01, -2.0085e-01],
            [ 1.5188e-01,  1.5188e-01,  1.5188e-01,  ..., -2.0085e-01,
             -2.00

In [180]:
for i in bo(features):
    print(i.shape)

torch.Size([12, 96, 1, 112, 140])
torch.Size([12, 192, 1, 56, 70])
torch.Size([12, 384, 1, 28, 35])
torch.Size([12, 768, 1, 14, 18])


In [3]:
[i - (5) + 1 for i in range(5)]

[-4, -3, -2, -1, 0]

In [3]:
import psutil
import numpy as np
total_memory = psutil.virtual_memory().total

In [2]:
total_memory

270230831104

In [4]:
arr = np.zeros((3,224, 280))

In [5]:
arr.nbytes

1505280

In [6]:
270230831104 / 1505280

179521.9700680272